<a href="https://colab.research.google.com/github/soleildayana/Planetary-Sciences/blob/main/OrbitasTNOs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desarrollo de interfaz para órbitas de Objetos Transneptunianos (TNOs)

## Soleil Dayana Niño Murcia


Desarrollado para la Tarea 9 de Ciencias Planetarias. Universidad de Antioquia


In [ ]:
# Librería
import numpy as np
import pandas as pd
import plotly.graph_objects as go

Se descarga un lote de datos desde el sitio del MPC, tomando el archivo daily_extended.json.gz con descripción Orbits for TNOs, Centaurs and SDOs. Luego, se sube a un repositorio en GitHub, para acceder a él desde el código a través de un URL.

Como el archivo contiene también órbitas de Centauros y SDOs, se deben filtrar los objetos transneptunianos. Esto, se hace bajo el criterio:

(a > 30 AU o q > 30 AU)

In [ ]:
# --- Acceder a los datos ---
url = "https://raw.githubusercontent.com/soleildayana/Astro/main/distant_extended.json.gz"
df = pd.read_json(url, compression="gzip")

# --- Filtrar TNOs ---
df["q"] = df["a"] * (1 - df["e"])
df_tno = df[df["q"] >= 30]
print("Cantidad de TNOs en el dataset:", len(df_tno))

# --- Función para xyz a partir de elementos orbitales ---
def get_xyz(a, e, i, Omega, omega, n_points=200):
    fs = np.linspace(0, 2*np.pi, n_points)
    r = a * (1 - e**2) / (1 + e * np.cos(fs))
    i, Omega, omega = np.radians([i, Omega, omega])
    x = r * (np.cos(Omega) * np.cos(omega + fs) - np.sin(Omega) * np.sin(omega + fs) * np.cos(i))
    y = r * (np.sin(Omega) * np.cos(omega + fs) + np.cos(Omega) * np.sin(omega + fs) * np.cos(i))
    z = r * (np.sin(i) * np.sin(omega + fs))
    return x, y, z

Cantidad de TNOs en el dataset: 4819


Creamos una función que permita visualizar de manera interactiva el sistema con TNOs a través de plotly

In [ ]:
# --- Función para graficar ---
def plot_tnos(df_sample, titulo):
    fig = go.Figure()
    for _, row in df_sample.iterrows():
        x, y, z = get_xyz(row['a'], row['e'], row['i'], row['Node'], row['Peri'])

        # Priorizar el nombre correcto
        if pd.notnull(row['Name']):
            trace_name = row['Name']
        elif pd.notnull(row['Principal_desig']):
            trace_name = row['Principal_desig']
        else:
            trace_name = str(row['Number'])

        fig.add_trace(go.Scatter3d(
            x=x, y=y, z=z,
            mode='lines',
            name=trace_name
        ))

    # Sol en el origen
    fig.add_trace(go.Scatter3d(
        x=[0], y=[0], z=[0],
        mode='markers',
        marker=dict(size=5, color='yellow'),
        name='Sun'
    ))

    fig.update_layout(
        scene=dict(xaxis_title='X (AU)', yaxis_title='Y (AU)', zaxis_title='Z (AU)'),
        width=800, height=700,
        template='plotly_dark',
        title=titulo
    )
    return fig

Visualicemos 100 órbitas de TNOs

In [ ]:
# --- Graficar 100 ---
fig_100 = plot_tnos(df_tno.sample(n=100, random_state=42), "Órbitas de 100 TNOs")
fig_100.show()

Ahora, para visualizar con mayor claridad, veamos únicamente 10 de las órbitas

In [ ]:
# ---Graficar 10---
fig_10 = plot_tnos(df_tno.sample(n=10, random_state=42), "Órbitas de 10 TNOs")
fig_10.show()